<a href="https://colab.research.google.com/github/MohammedHamood/20NewsGroup/blob/main/20newsdataset_DCdistance_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform DCdicetance alorithm for all models assuming no need for regularization and no need for early stoping as the number of features are very low equals to k.

In [ ]:
## Loading dataBase and initializtion process

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin

import time
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

import pipelines_FEngineering as BaseLine
from sklearn.datasets import fetch_20newsgroups
import preprocessingNLP as PNLP

from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA, NMF,TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import Normalizer

# Import Dataset
TNGD_Train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])
TNGD_test = fetch_20newsgroups(subset='test',shuffle=True, random_state=42,remove=['headers', 'footers', 'quotes'])

# Preprocessing
print("PREPROCESSING ...")
TNGD_Train.data = PNLP.customNLP(TNGD_Train.data)
TNGD_test.data = PNLP.customNLP(TNGD_test.data)
TNGD_Train.data, TNGD_Train.target = PNLP.removeEmptyInstances(TNGD_Train.data, TNGD_Train.target)
resample=np.linspace(0,TNGD_Train.target.size,TNGD_Train.target.size,endpoint=False, retstep=False, dtype=int)

ros = RandomOverSampler(random_state=42)
resample, TNGD_Train.target =ros.fit_resample(resample[:,None], TNGD_Train.target)
data=[]
for i in resample:
    data.append(TNGD_Train.data[int(i)])
TNGD_Train.data=data
data=[]
resample=[] 


print("PREPROCESSING DONE!")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


PREPROCESSING ...


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


PREPROCESSING DONE!


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# needed functions 

In [ ]:

# Utility function to report best scores
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("Mean Fit Time: %.3f seconds" %results['mean_fit_time'][candidate])
            print("")


from scipy import sparse
def Pipeline_FeatureEngineering(train_data,train_target,parameters=None,CV=10, 
                                reductionMethod=PCA(),reductionType=1,model=LinearSVC()):
    class DenseTransformer(TransformerMixin):

        def fit(self, X, y=None, **fit_params):
            return self

        def transform(self, X, y=None, **fit_params):
            return X.todense()
    class sparseTransformer(TransformerMixin):

        def fit(self, X, y=None, **fit_params):
            return self

        def transform(self, X, y=None, **fit_params):
            return sparse.csr_matrix(X) 
        
 

    class dcdistanceTransformer(TransformerMixin, BaseEstimator): 
        """
        proposed method modifed from(DCDistance: A Supervised Text Document Feature extraction based on class labels)
        """
        def __init__(self):
            super(dcdistanceTransformer, self).__init__()
            

        def fit(self, X, y, **fit_params):
            self.alpha=20
            self.Classes=np.unique(y, return_index = False)
            self.split=np.linspace(0, X.shape[1],self.alpha,endpoint = False,  dtype=int)
            self.vd=np.zeros((len(self.Classes),X.shape[1]))
            self.vd2=np.zeros((len(self.Classes),X.shape[1]))
            for i in self.Classes:
                ind = np.where(y == i)[0]
                temp=np.zeros((1,X.shape[1]))
                for feat in range(0,len(self.split)):
                    ind1=self.split[feat]              
                    ind2=self.split[feat]+self.split[1]
                    temp=X.tocsr()[ind,ind1:ind2].sum(axis = 0)
                    self.vd2[int(i),ind1:ind2]=temp     
            tfidf_transformer = TfidfTransformer()
            self.vd2 = tfidf_transformer.fit_transform(self.vd2) 
            self.vd2=self.vd2.todense()   
            return self

        def transform(self, X, **fit_params):
            V= np.zeros((X.shape[0],len(self.Classes)))

            
            for i in self.Classes:
                starting=0
                ending=200
                while starting<X.shape[0]-2:
                    if ending>X.shape[0]:
                        ending=X.shape[0]
                    temp=np.linalg.norm(X.tocsr()[starting:ending,:]-self.vd2[int(i),:],axis=1,ord=2)
                    V[starting:ending,int(i)]= temp[:] #('Norm',Normalizer(copy=False)),
                    starting=ending
                    ending=ending+200
            V= sparse.csr_matrix(V)
            X=[]
            return V         
#
        
    if reductionType==1:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer()),
                             ('redu',dcdistanceTransformer()),
                             ('Norm',Normalizer(copy=False)),('clf', model)])
    elif reductionType==2:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer()),
                             ('redu',reductionMethod),
                             ('Norm',Normalizer(copy=False)),('clf', model)])
    elif reductionType==3:
        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),('tfidf', TfidfTransformer()),
                             ('Norm1',Normalizer(copy=False)),('clf', model)])    
    elif reductionType==0:
        text_clf = Pipeline([('vect', CountVectorizer()),('samp',ADASYN(random_state=42)),('tfidf', TfidfTransformer()),
                             ('redu1',TruncatedSVD(n_components=300)),('Norm1',Normalizer(copy=False)),('redu',reductionMethod),
                             ('Norm2',Normalizer(copy=False)),('clf', model)])
    gs_clf = GridSearchCV(text_clf, parameters, cv=CV, n_jobs=-1)
    gs_clf = gs_clf.fit(train_data, train_target)
    return gs_clf


In [ ]:
#setting defined parameters
ngram_range=(1,2)
cv_folds=11

In [ ]:
#1)logistic regression

In [ ]:
# Set relevant parameters
parameters = {
     'clf__C': [2000],
     #'clf__tol': (0.0001, 0.001)
}

# Initialize gridsearchcv
start_time = time.perf_counter()
gs_clf= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=LogisticRegression(solver='saga',max_iter=1000))
elapsed_time = time.perf_counter() - start_time

# Print results
print("Validation Accuracy: " + str(gs_clf.best_score_))
print("Optimal Parameters: " + str(gs_clf.best_estimator_))
print("Optimal Parameters: " + str(gs_clf.best_params_))
print("")

train_accuracies = gs_clf.cv_results_['mean_test_score']
train_time = gs_clf.cv_results_['mean_fit_time']
train_params = gs_clf.cv_results_['params']

for i in range(len(train_time)):
  print("Parameter: {0}".format(train_params[i]))
  print("Training Time: %.3f seconds" %train_time[i])
  print("Valdation Accuracy: {0:.3f}".format(train_accuracies[i]))
  print("")



In [ ]:
#2)SVM

In [ ]:
parameters = {
     'clf__C': [2000],
     #'clf__tol': (0.0001, 0.001)
}

# Initialize gridsearchcv
start_time = time.perf_counter()
gs_clf= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=LinearSVC(max_iter=1000))
elapsed_time = time.perf_counter() - start_time

# Print results
print("Validation Accuracy: " + str(gs_clf.best_score_))
print("Optimal Parameters: " + str(gs_clf.best_estimator_))
print("Optimal Parameters: " + str(gs_clf.best_params_))
print("")

train_accuracies = gs_clf.cv_results_['mean_test_score']
train_time = gs_clf.cv_results_['mean_fit_time']
train_params = gs_clf.cv_results_['params']

for i in range(len(train_time)):
  print("Parameter: {0}".format(train_params[i]))
  print("Training Time: %.3f seconds" %train_time[i])
  print("Valdation Accuracy: {0:.3f}".format(train_accuracies[i]))
  print("")

KeyboardInterrupt: ignored

In [ ]:
cv_folds=10
# Set relevant parameters
parameters = {
    'clf__min_samples_split': [100],
    #'clf__max_features': ('sqrt', 'log2'),
    #'clf__n_estimators': (80,200),
    }

# Initialize gridsearchcv
start_time = time.perf_counter()
gs_clf= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=LinearSVC(max_iter=1000))
elapsed_time = time.perf_counter() - start_time

# Print results
print("Validation Accuracy: " + str(gs_clf.best_score_))
print("Optimal Parameters: " + str(gs_clf.best_estimator_))
print("Optimal Parameters: " + str(gs_clf.best_params_))
print("")

train_accuracies = gs_clf.cv_results_['mean_test_score']
train_time = gs_clf.cv_results_['mean_fit_time']
train_params = gs_clf.cv_results_['params']

for i in range(len(train_time)):
  print("Parameter: {0}".format(train_params[i]))
  print("Training Time: %.3f seconds" %train_time[i])
  print("Valdation Accuracy: {0:.3f}".format(train_accuracies[i]))
  print("")

In [ ]:
parameters = {
    'clf__n_estimators': (8,50,80),
    'clf__base_estimator':[LogisticRegression(solver='saga',max_iter=1000)],
    'clf__base_estimator_C':(1.0,5,10, 100.0,1000,50),
    }

# Initialize gridsearchcv
ada_LR_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=AdaBoostClassifier())

# Execute gridsearchcv and print best results 

report(ada_LR_base.cv_results_)


test_pridect=ada_LR_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))

In [ ]:
parameters = {
    'clf__n_estimators': (8,50,80),
    'clf__base_estimator':[LinearSVC()],
    'clf__base_estimator_C':(1.0,5,10, 100.0,1000,50),
    }

# Initialize gridsearchcv
ada_LR_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=AdaBoostClassifier())

# Execute gridsearchcv and print best results 

report(ada_LR_base.cv_results_)


test_pridect=ada_LR_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))

In [ ]:
parameters = {
    #'clf__min_samples_split': (200,100),
    #'clf__max_features': ('sqrt', 'log2'),
    'clf__n_estimators': (10,50,100),
    }

# Initialize gridsearchcv
RF_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=RandomForestClassifier(max_features='sqrt'))

# Execute gridsearchcv and print best results 

report(RF_base.cv_results_)


test_pridect=RF_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))

In [ ]:
parameters = {
    'clf__n_estimators': (8,50,80),
    'clf__base_estimator':[RandomForestClassifier(max_features='sqrt')],
    }

# Initialize gridsearchcv
ada_RF_base= Pipeline_FeatureEngineering(TNGD_Train.data,TNGD_Train.target,parameters,CV=cv_folds
                                         , reductionMethod=PCA(),reductionType=1,model=AdaBoostClassifier())

# Execute gridsearchcv and print best results 

report(ada_RF_base.cv_results_)


test_pridect=ada_RF_base.predict(TNGD_test.data)
print(classification_report(test_pridect, TNGD_test.target))

print(np.mean(test_pridect==TNGD_test.target))